In [2]:
# --- Config (edit these if needed) ---
weights_path = "weights/best.pt"           # put your weight file here after download
video_path   = "assets/Traffic Control CCTV.mp4"       # add a small sample or tell users to provide their own
conf_thresh  = 0.5

# Optional: gate training so "Run All" won't accidentally start
RUN_TRAINING = False


In [ ]:
"""
Training
- I trained locally for 20 epochs to validate the pipeline.
- Final 100-epoch training was done by my supervisor on a GPU with ~3000 images.
- This cell is OFF by default to avoid accidental runs and dataset path issues.
"""
if RUN_TRAINING:
    from ultralytics import YOLO
    # use a relative/placeholder path; don't hardcode D:/...
    data_yaml = "data/data.yaml"  # <-- tell users how to prepare this if they have the dataset
    model = YOLO("yolov8n.pt")    # base model
    model.train(data=data_yaml, epochs=20)


In [6]:
import cv2
import cvzone
import math
import os
from ultralytics import YOLO
from paddleocr import PaddleOCR

# Check if we're running from notebooks folder and adjust paths
if os.path.basename(os.getcwd()) == 'notebooks':
    weights_path = "../" + weights_path
    video_path = "../" + video_path

# Debug: Print paths to verify
print(f"Current directory: {os.getcwd()}")
print(f"Weights path: {weights_path}")
print(f"Video path: {video_path}")
print(f"Weights exists: {os.path.exists(weights_path)}")

# Use webcam if video file doesn't exist
if not os.path.exists(video_path):
    print("Video file not found, using webcam...")
    video_path = 0

# Initialize OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Video path
cap = cv2.VideoCapture(video_path)

# Load YOLO model
model = YOLO(weights_path)
classnames = ['license-plate', 'vehicle']

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 model
    frame = cv2.resize(frame, (1080, 720))
    results = model(frame, verbose=False)

    for result in results:
        boxes = result.boxes
        if boxes is not None:  # Add safety check
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                # Add bounds check to prevent index errors
                if conf > conf_thresh and cls < len(classnames) and classnames[cls] == 'license-plate':
                    # Crop the license plate
                    plate_crop = frame[y1:y2, x1:x2]

                    # Only run OCR if crop is valid
                    plate_text = "Reading..."
                    if plate_crop.size > 0:
                        try:
                            ocr_result = ocr.ocr(plate_crop, cls=True)
                            if ocr_result and ocr_result[0] and isinstance(ocr_result[0], list):
                                plate_text = ' '.join([line[1][0] for line in ocr_result[0] if line[1][0].strip()])
                                if not plate_text.strip():
                                    plate_text = "Unreadable"
                            else:
                                plate_text = "No text detected"
                        except Exception as e:
                            plate_text = "OCR Error"
                            print(f"OCR Error: {e}")

                    # Draw rectangle and text
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cvzone.putTextRect(frame, plate_text, [x1 + 5, y1 - 10], scale=1, thickness=2, colorR=(0, 255, 0))

    # Show the frame
    cv2.imshow("License Plate Recognition", frame)

    # Press 't' to stop
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

TypeError: can only concatenate str (not "int") to str

In [12]:
import cv2
import cvzone
import math
import os
import csv
import datetime
from ultralytics import YOLO
from paddleocr import PaddleOCR
import pandas as pd
from collections import defaultdict
import time

# Get fresh variables from the config (avoid accumulated path issues)
weights_path = "weights/best.pt"
video_path = "assets/Traffic Control CCTV.mp4"
conf_thresh = 0.5

# Check if we're running from notebooks folder and adjust paths
if os.path.basename(os.getcwd()) == 'notebooks':
    weights_path = "../" + weights_path
    if isinstance(video_path, str):  # Only adjust if it's a string path
        video_path = "../" + video_path

# Debug: Print paths to verify
print(f"Current directory: {os.getcwd()}")
print(f"Weights path: {weights_path}")
print(f"Video path: {video_path}")
print(f"Weights exists: {os.path.exists(weights_path)}")

# Check video path existence only if it's a file path
if isinstance(video_path, str):
    print(f"Video exists: {os.path.exists(video_path)}")
else:
    print("Using webcam (video_path = 0)")

# Create results directory if it doesn't exist
results_dir = "../results" if os.path.basename(os.getcwd()) == 'notebooks' else "results"
os.makedirs(results_dir, exist_ok=True)

# Initialize CSV file with headers
csv_filename = os.path.join(results_dir, f"license_plates_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
csv_headers = ['timestamp', 'frame_number', 'plate_text', 'confidence', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']

# Initialize tracking variables
frame_count = 0
detected_plates = []
plate_tracker = defaultdict(int)
last_save_time = time.time()
save_interval = 30

# Initialize OCR
print("🔧 Initializing PaddleOCR...")
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load YOLO model first
print("🤖 Loading YOLO model...")

# Verify the weights path before loading
if not os.path.exists(weights_path):
    print(f"❌ Model weights not found at: {weights_path}")
    print("Available options:")
    print("1. Download best.pt from GitHub releases")
    print("2. Use a pretrained YOLOv8 model temporarily")
    
    choice = input("Use YOLOv8n temporarily? (y/n): ")
    if choice.lower() == 'y':
        weights_path = "yolov8n.pt"  # This will download automatically
        print("📥 Using YOLOv8n (general object detection)")
    else:
        print("❌ Exiting...")
        exit()

model = YOLO(weights_path)
classnames = ['license-plate', 'vehicle']

# Handle video source
if isinstance(video_path, str) and not os.path.exists(video_path):
    print("⚠️ Video file not found! Available options:")
    print("1. Use webcam (press 1)")
    print("2. Provide different video path (press 2)")
    print("3. Exit (press any other key)")
    
    choice = input("Enter your choice: ")
    
    if choice == "1":
        video_path = 0  # Webcam
        print("📹 Using webcam...")
    elif choice == "2":
        video_path = input("Enter video file path: ")
        if not os.path.exists(video_path):
            print("❌ File still not found. Exiting...")
            exit()
    else:
        print("❌ Exiting...")
        exit()

# Video capture
print(f"📹 Opening video source: {video_path}")
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Error: Cannot open video source")
    print("Try:")
    print("1. Check if video file exists")
    print("2. Use webcam by setting video_path = 0")
    print("3. Check video file format (MP4, AVI, etc.)")
    exit()

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS)) if video_path != 0 else 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if video_path != 0 else 0

print(f"📹 Video Info: {frame_width}x{frame_height} @ {fps}fps")
if total_frames > 0:
    print(f"📊 Total frames: {total_frames}")

# Test if we can read a frame
ret, test_frame = cap.read()
if not ret:
    print("❌ Cannot read first frame from video")
    cap.release()
    exit()
else:
    print("✅ Successfully read first frame")
    # Reset video to beginning
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Function to clean and validate plate text
def clean_plate_text(text):
    """Clean and validate extracted plate text"""
    if not text or text in ["Reading...", "OCR Error", "No text detected", "Unreadable"]:
        return None
    
    # Remove special characters and clean text
    cleaned = ''.join(char.upper() for char in text if char.isalnum() or char.isspace())
    cleaned = ' '.join(cleaned.split())
    
    # Basic validation
    if len(cleaned) < 2 or len(cleaned) > 15:
        return None
    
    return cleaned

# Function to save data to CSV (immediate write)
def save_detection_immediately(detection_data, filename):
    """Save single detection to CSV immediately"""
    try:
        file_exists = os.path.exists(filename)
        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(csv_headers)  # Write headers if new file
            writer.writerow(detection_data)
        return True
    except Exception as e:
        print(f"❌ Error saving to CSV: {e}")
        return False

# Function to save all data to CSV
def save_to_csv(data, filename):
    """Save all detection data to CSV file"""
    try:
        df = pd.DataFrame(data, columns=csv_headers)
        df.to_csv(filename, index=False)
        print(f"💾 Saved {len(data)} detections to {filename}")
    except Exception as e:
        print(f"❌ Error saving CSV: {e}")

print("🚀 Starting detection... Press 't' to stop, 's' to save current results")
print("📝 Detections will be saved immediately to CSV")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            if video_path == 0:
                print("📹 Cannot read from webcam")
                break
            else:
                print("📹 End of video")
                break

        frame_count += 1
        current_time = datetime.datetime.now()

        # Resize frame for consistent processing
        original_frame = frame.copy()
        frame = cv2.resize(frame, (1080, 720))
        
        # Run YOLOv8 model
        results = model(frame, verbose=False)

        # Process detections
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cls = int(box.cls[0])

                    # Process both license plates and vehicles (if using custom model)
                    if conf > conf_thresh and cls < len(classnames):
                        class_name = classnames[cls]
                        
                        # Draw all detections
                        if class_name == 'license-plate':
                            color = (0, 255, 0)  # Green for plates
                            
                            # Crop the license plate with padding
                            padding = 5
                            x1_crop = max(0, x1 - padding)
                            y1_crop = max(0, y1 - padding)
                            x2_crop = min(frame.shape[1], x2 + padding)
                            y2_crop = min(frame.shape[0], y2 + padding)
                            
                            plate_crop = frame[y1_crop:y2_crop, x1_crop:x2_crop]

                            # Run OCR
                            plate_text = "Processing..."
                            if plate_crop.size > 0:
                                try:
                                    ocr_result = ocr.ocr(plate_crop, cls=True)
                                    if ocr_result and ocr_result[0] and isinstance(ocr_result[0], list):
                                        raw_text = ' '.join([line[1][0] for line in ocr_result[0] if line[1][0].strip()])
                                        cleaned_text = clean_plate_text(raw_text)
                                        
                                        if cleaned_text:
                                            plate_text = cleaned_text
                                            plate_tracker[plate_text] += 1
                                            
                                            # Store detection data
                                            detection_data = [
                                                current_time.strftime('%Y-%m-%d %H:%M:%S'),
                                                frame_count,
                                                plate_text,
                                                round(conf, 3),
                                                x1, y1, x2, y2
                                            ]
                                            detected_plates.append(detection_data)
                                            
                                            # Save immediately to CSV
                                            save_detection_immediately(detection_data, csv_filename)
                                            
                                            display_text = f"{plate_text} ({conf:.2f})"
                                            print(f"🎯 Detected: {plate_text} (conf: {conf:.2f})")
                                        else:
                                            display_text = f"Unreadable ({conf:.2f})"
                                    else:
                                        display_text = f"No text ({conf:.2f})"
                                except Exception as e:
                                    display_text = f"OCR Error ({conf:.2f})"
                                    print(f"OCR Error: {e}")
                            else:
                                display_text = f"Invalid crop ({conf:.2f})"
                        
                        elif class_name == 'vehicle':
                            color = (255, 0, 0)  # Blue for vehicles
                            display_text = f"Vehicle ({conf:.2f})"
                        
                        else:
                            # For YOLOv8n general model, show all detected objects
                            color = (0, 255, 255)  # Yellow for other objects
                            display_text = f"Object ({conf:.2f})"
                        
                        # Draw bounding box and text
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cvzone.putTextRect(frame, display_text, [x1 + 5, y1 - 10], 
                                         scale=0.7, thickness=1, colorR=color)

        # Add frame info overlay
        info_text = f"Frame: {frame_count} | Plates: {len(detected_plates)}"
        if total_frames > 0:
            progress = (frame_count / total_frames) * 100
            info_text += f" | Progress: {progress:.1f}%"
        
        cvzone.putTextRect(frame, info_text, [10, 30], scale=0.8, thickness=1, colorR=(255, 255, 255))

        # Show the frame
        cv2.imshow("License Plate Recognition", frame)

        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('t'):  # Stop
            break
        elif key == ord('s'):  # Manual save summary
            if detected_plates:
                save_to_csv(detected_plates, csv_filename.replace('.csv', '_summary.csv'))
                print(f"📊 Summary saved with {len(detected_plates)} detections")
            else:
                print("⚠️ No detections to save")

except KeyboardInterrupt:
    print("\n🛑 Interrupted by user")
except Exception as e:
    print(f"❌ Error during processing: {e}")

finally:
    # Final save and cleanup
    if detected_plates:
        save_to_csv(detected_plates, csv_filename.replace('.csv', '_final.csv'))
        
        # Print summary statistics
        print(f"\n📊 Detection Summary:")
        print(f"Total detections: {len(detected_plates)}")
        print(f"Unique plates: {len(plate_tracker)}")
        print(f"CSV saved to: {csv_filename}")
        
        # Show most frequent plates
        if plate_tracker:
            print("\n🏆 Most detected plates:")
            sorted_plates = sorted(plate_tracker.items(), key=lambda x: x[1], reverse=True)
            for plate, count in sorted_plates[:5]:
                print(f"  {plate}: {count} times")
    else:
        print("⚠️ No license plates detected")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    print("🧹 Cleanup completed")

Current directory: d:\license-plate-recognition\notebooks
Weights path: ../weights/best.pt
Video path: ../assets/Traffic Control CCTV.mp4
Weights exists: True
Video exists: True
🔧 Initializing PaddleOCR...
🤖 Loading YOLO model...
📹 Opening video source: ../assets/Traffic Control CCTV.mp4
📹 Video Info: 3840x2160 @ 30fps
📊 Total frames: 1800
✅ Successfully read first frame
🚀 Starting detection... Press 't' to stop, 's' to save current results
📝 Detections will be saved immediately to CSV
🎯 Detected: MOSIVSU (conf: 0.77)
🎯 Detected: LOSAISAM (conf: 0.75)
🎯 Detected: MMSIVSU (conf: 0.71)
🎯 Detected: NAI3 NRU (conf: 0.75)
🎯 Detected: MOSIVSU (conf: 0.74)
🎯 Detected: MYSIVSU (conf: 0.75)
🎯 Detected: NAI3 NRU (conf: 0.72)
🎯 Detected: MMSIVSU (conf: 0.78)
🎯 Detected: MMSIVSU (conf: 0.79)
🎯 Detected: GXISOCJ (conf: 0.53)
🎯 Detected: NSAISAN (conf: 0.78)
🎯 Detected: CXI5OCJ (conf: 0.57)
🎯 Detected: MSAISAN (conf: 0.75)
🎯 Detected: GXI5OGJ (conf: 0.56)
🎯 Detected: MMSIVSU (conf: 0.72)
🎯 Detected: